# Automação Web e Busca de Informações com Python e Selenium


In [21]:
#1) Entrar no site que estamos buscando (https://google.com.br);
#2) No campo de pesquisa, digitar “cotação dólar”;
#3) Apertar ENTER para Pesquisar;
#4) Pegar a cotação indicada

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# abrir um navegador
navegador = webdriver.Chrome("chromedriver.exe")

navegador.get("https://www.google.com/")

#pegar a cotação do dolar
navegador.find_element(By.XPATH,
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação dolar")
navegador.find_element(By.XPATH,
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER)
cotacao_dolar = navegador.find_element(By.XPATH,
                        '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#pegar a cotação do euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys("cotação euro")
navegador.find_element(By.XPATH,
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input").send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH,
                        '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

#pegar a cotação do ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")


navegador.quit()


print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)


C:\Users\ryus4\AppData\Local\Temp/ipykernel_47740/1111818785.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  navegador = webdriver.Chrome("chromedriver.exe")


4.8349
5.322016175
305.17


###  Atualizar a base de preços com as novas cotações

- Importando a base de dados

In [23]:
import pandas as pd

tabela = pd.read_excel("produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [24]:
# Passo 5: Recalcular o preço de cada produto
# atualizar a cotação
# nas linhas onde na coluna "Moeda" = Dólar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# atualizar o preço base reais (preço base original * cotação)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# atualizar o preço final (preço base reais * Margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

# tabela["Preço de Venda"] = tabela["Preço de Venda"].map("R${:.2f}".format)

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,4.834900,4834.851651,1.40,6768.792311
1,Carro Renault,4500.00,Euro,5.322016,23949.072788,2.00,47898.145575
2,Notebook Dell,899.99,Dólar,4.834900,4351.361651,1.70,7397.314807
3,IPhone,799.00,Dólar,4.834900,3863.085100,1.70,6567.244670
4,Carro Fiat,3000.00,Euro,5.322016,15966.048525,1.90,30335.492197
5,Celular Xiaomi,480.48,Dólar,4.834900,2323.072752,2.00,4646.145504
6,Joia 20g,20.00,Ouro,305.170000,6103.400000,1.15,7018.910000


### Agora vamos exportar a nova base de preços atualizada

In [25]:
# Passo 6: Salvar os novos preços dos produtos
tabela.to_excel("Produtos Novo.xlsx", index=False)